# <u> **Player Subscription Final Report**


By Xuan Tung Duong, Sara Garcia Rubiera, Daniel Samari and Demelza Awogu


### <u> **Introduction**

The Pacific Laboratory for Artificial Intelligence (PLAI) is a research group at UBC collecting data about how people play video games. In this particular project, MineCraft serverLinks were set up to an external site where players' actions are recorded as they engage with the game [_**INSERT REFERENCE**_]. The issue that arises is that they want to make sure that they have enough resources to ensure optimal experiences for players. A research question to we want to answer is:

_Question 1: What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?_

In this project, we wanted to answer a more specified research question from the one above, and hence our research question is as follows:

**Can we predict the  the likelihood of a player subscribing to the game newsletter based on their age, and their experience level?**



### <u> **Data Description**

In the table below, there are 9 columns and 196 rows.
The variables describe the following: 

1.`experience` is a categorical variable, with levels of experience, where `Veteran` is the highest, followed by `Pro`, `Regular` and `Amateur`.

2. `subscribe` is a booleen variable, indicating the whether players have subscribed to the newsletter of the game.

3. `hashedEmail` is an ID variable that refers to the players email.

4. `played_hours` is a quantitiative variable - refers to the number of hours played on the game.

5. `name` refers to the name of the player.

6. `gender` indicates that of the player, and `age` is their age.

7. 

In [1]:
#Input libraries 
import pandas as pd 
import numpy as np 
import altair as alt 

In [2]:
#Import urls
players_url = "https://drive.google.com/uc?id=1Mw9vW0hjTJwRWx0bDXiSpYsO3gKogaPz"
players = pd.read_csv(players_url)
players

,experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName
0,Pro,True,f6daba428a5e19a3d47574858c13550499be23603422e6...,30.3,Morgan,Male,9,NaN,NaN
1,Veteran,True,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...,3.8,Christian,Male,17,NaN,NaN
2,Veteran,False,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...,0.0,Blake,Male,17,NaN,NaN
3,Amateur,True,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...,0.7,Flora,Female,21,NaN,NaN
4,Regular,True,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...,0.1,Kylie,Male,21,NaN,NaN
...,...,...,...,...,...,...,...,...,...
191,Amateur,True,b6e9e593b9ec51c5e335457341c324c34a2239531e1890...,0.0,Bailey,Female,17,NaN,NaN
192,Veteran,False,71453e425f07d10da4fa2b349c83e73ccdf0fb3312f778...,0.3,Pascal,Male,22,NaN,NaN
193,Amateur,False,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db29...,0.0,Dylan,Prefer not to say,17,NaN,NaN
194,Amateur,False,f19e136ddde68f365afc860c725ccff54307dedd13968e...,2.3,Harlow,Male,17,NaN,NaN


### <u> **Methods and Results**

In [3]:
# load libraries
import numpy as np
import pandas as pd
import altair as alt
from sklearn import set_config
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

set_config(transform_output="pandas")

# set random seed
np.random.seed(100)

In [4]:
# clean up dataframe
players.drop(columns=['individualId','organizationName'],inplace = True)

players

,experience,subscribe,hashedEmail,played_hours,name,gender,age
0,Pro,True,f6daba428a5e19a3d47574858c13550499be23603422e6...,30.3,Morgan,Male,9
1,Veteran,True,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...,3.8,Christian,Male,17
2,Veteran,False,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...,0.0,Blake,Male,17
3,Amateur,True,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...,0.7,Flora,Female,21
4,Regular,True,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...,0.1,Kylie,Male,21
...,...,...,...,...,...,...,...
191,Amateur,True,b6e9e593b9ec51c5e335457341c324c34a2239531e1890...,0.0,Bailey,Female,17
192,Veteran,False,71453e425f07d10da4fa2b349c83e73ccdf0fb3312f778...,0.3,Pascal,Male,22
193,Amateur,False,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db29...,0.0,Dylan,Prefer not to say,17
194,Amateur,False,f19e136ddde68f365afc860c725ccff54307dedd13968e...,2.3,Harlow,Male,17


In [5]:
# assign experience numeric values
# 0-4 scale, with 0 being least experienced and 4 being most experienced
experience_value = {'Beginner':0,'Amateur':1,'Regular':2,'Veteran':3,'Pro':4}

experience_to_val = players['experience'].to_numpy().copy()

for old_val, new_val in experience_value.items():
    experience_to_val[experience_to_val == old_val] = new_val

players = players.assign(experience_value=experience_to_val)
players

,experience,subscribe,hashedEmail,played_hours,name,gender,age,experience_value
0,Pro,True,f6daba428a5e19a3d47574858c13550499be23603422e6...,30.3,Morgan,Male,9,4
1,Veteran,True,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...,3.8,Christian,Male,17,3
2,Veteran,False,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...,0.0,Blake,Male,17,3
3,Amateur,True,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...,0.7,Flora,Female,21,1
4,Regular,True,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...,0.1,Kylie,Male,21,2
...,...,...,...,...,...,...,...,...
191,Amateur,True,b6e9e593b9ec51c5e335457341c324c34a2239531e1890...,0.0,Bailey,Female,17,1
192,Veteran,False,71453e425f07d10da4fa2b349c83e73ccdf0fb3312f778...,0.3,Pascal,Male,22,3
193,Amateur,False,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db29...,0.0,Dylan,Prefer not to say,17,1
194,Amateur,False,f19e136ddde68f365afc860c725ccff54307dedd13968e...,2.3,Harlow,Male,17,1


In [6]:
# split the data into training and testing sets
# we use a split of 80/20 because the data set is small
# this leaves 40 values in the test set and 156 in the training set.
# ensuring there is enough data in the test set to provide an accurate measure of fit
# while still keeping the training set large enough to properly train the model
players_train, players_test = train_test_split(
    players, train_size=0.80, stratify=players['subscribe']
)
players_train

,experience,subscribe,hashedEmail,played_hours,name,gender,age,experience_value
163,Regular,True,d43af3bed5e9f1f31077233697c18f3f988a217bd0376a...,0.5,Xia,Female,20,2
19,Regular,True,20ae3f8b5323674fe6e537bc91928841cfc2a00c1b3652...,0.6,Ryker,Male,19,2
121,Beginner,True,9c2a5275ada716508a7e66629dfbeaa4c03ca9eb5f30d2...,0.1,Bella,Male,24,0
10,Veteran,True,7a4686586d290c67179275c7c3dfb4ea02f4d317d9ee0e...,1.6,Lane,Female,23,3
153,Beginner,True,0ce7bfa910d47fc91f21a7b3acd8f33bde6db57912ce02...,0.1,Osiris,Male,17,0
...,...,...,...,...,...,...,...,...
195,Pro,True,d9473710057f7d42f36570f0be83817a4eea614029ff90...,0.2,Ahmed,Other,91,4
53,Amateur,True,fc0224c81384770e93ca717f32713960144bf0b52ff676...,0.2,Gemna,Male,27,1
130,Amateur,True,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,56.1,Dana,Male,23,1
72,Veteran,True,1edebbf13898dbe99da5ee743b1fd9cbfdc6d69f1180dd...,0.0,Will,Male,17,3


In [7]:
# visualise variables of interest
age_dist = alt.Chart(players_train).mark_bar().encode(
    x = alt.X('age').bin(step=5).title('Age'),
    y = alt.Y('count()').title('Number of Players'),
    color = alt.Color('subscribe').title('Subcribed to newsletter?')
).properties(
    title=['Panel A: Player Age Distribution']
)

exp_bar = alt.Chart(players_train).mark_bar().encode(
    x = alt.X('experience').title('Experience Level'),
    y = alt.Y('count()').title('Number of Players'),
    color = alt.Color('subscribe').title('Subcribed to newsletter?')
).properties(
    title=['Panel C: Player Experience Distribution']
)

time_dist = alt.Chart(players_train).mark_bar().encode(
    x = alt.X('played_hours').bin(step=10).title('Time Played (hrs)'),
    y = alt.Y('count()').title('Proportion of Players'),
    color = alt.Color('subscribe').title('Subcribed to newsletter?')
).properties(
    title=['Pancel B: Panel Player Play Time Distribution']
)

gender_bar = alt.Chart(players_train).mark_bar().encode(
    x = alt.X('gender').title('Gender'),
    y = alt.Y('count()').title('Number of Players'),
    color = alt.Color('subscribe').title('Subcribed to newsletter?')
).properties(
    title=['Panel D: Player Gender Distribution']
)

alt.vconcat(alt.hconcat(age_dist, time_dist), alt.hconcat(exp_bar, gender_bar)).properties(title=['Fig 1. Visualisation of Player Observations in the Training Set','Bars Split by Subscription Status'])

alt.VConcatChart(...)

In [8]:
age_v_time = alt.Chart(players_train).mark_point().encode(
    y = alt.Y('played_hours').title('Hours Played'),
    x = alt.X('age').title('Age'),
    color = alt.Color('subscribe').title('Subcribed to newsletter?')
).properties(
    title=['Panel A: Plot of Hours Played vs. Age of Players']
)

exp_v_time = alt.Chart(players_train).mark_point().encode(
    y = alt.Y('played_hours').title('Hours Played'),
    x = alt.X('experience_value').title('Level of Experience'),
    color = alt.Color('subscribe').title('Subcribed to newsletter?')
).properties(
    title=['Panel B: Plot of Hours Played vs. Experience Level']
)

alt.hconcat(age_v_time, exp_v_time).properties(title=['Fig 2. Exploratory Plots of the Relationship between Selected Variables'])

alt.HConcatChart(...)

In [9]:
# check for class imbalance
players_train['subscribe'].value_counts()

subscribe
True     115
False     41
Name: count, dtype: int64

In [10]:
# oversample 'False' class to offset class imbalance
not_sub = players_train[players_train['subscribe'] == False]
subbed = players_train[players_train['subscribe'] == True]
not_sub_upsample = not_sub.sample(
    n=subbed.shape[0], replace=True
)
players_train_upsample = pd.concat((not_sub_upsample, subbed))
players_train_upsample["subscribe"].value_counts()

subscribe
False    115
True     115
Name: count, dtype: int64

In [11]:
# preprocess the data by standardising the numeric variables
players_preprocessor = make_column_transformer(
    (StandardScaler(), ['played_hours', 'age','experience_value']),
    remainder='passthrough',
    verbose_feature_names_out=False
)

# make the pipeline
players_pipe = make_pipeline(players_preprocessor, KNeighborsClassifier())

# define the range of n_neighbours for optimisation
param_grid = { 'kneighborsclassifier__n_neighbors': range(1,31,1) }

# perform a gridsearch to find best k-value
# we cross validate only 5 times because of the small amount of data
# more cross validations would provide lower variance in results but are more subject to bias due to small data sets
# while less cross validations are less subject to bias but have higher variance
players_search = GridSearchCV(
    estimator=players_pipe,
    param_grid=param_grid,
    cv=5,
    return_train_score=True,
    n_jobs=-1
)

# fit the model
players_search.fit(
    players_train_upsample[['played_hours', 'age','experience_value']],
    players_train_upsample['subscribe']
)

# output the results into a dataframe
cv_results = pd.DataFrame(players_search.cv_results_)
cv_results.sort_values(by='rank_test_score').head(5).reset_index()

,index,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kneighborsclassifier__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0,0.007578,0.005373,0.005320,0.000273,1,{'kneighborsclassifier__n_neighbors': 1},0.869565,0.804348,0.739130,...,0.773913,0.056018,1,0.875000,0.891304,0.913043,0.896739,0.918478,0.898913,0.015601
1,2,0.004407,0.000061,0.005139,0.000019,3,{'kneighborsclassifier__n_neighbors': 3},0.804348,0.717391,0.695652,...,0.726087,0.052535,2,0.782609,0.826087,0.853261,0.826087,0.842391,0.826087,0.024061
2,1,0.004474,0.000064,0.005130,0.000030,2,{'kneighborsclassifier__n_neighbors': 2},0.739130,0.760870,0.673913,...,0.726087,0.029488,3,0.766304,0.798913,0.836957,0.820652,0.836957,0.811957,0.026758
3,6,0.004393,0.000058,0.005150,0.000008,7,{'kneighborsclassifier__n_neighbors': 7},0.847826,0.717391,0.608696,...,0.708696,0.078502,4,0.733696,0.739130,0.782609,0.744565,0.744565,0.748913,0.017323
4,3,0.004411,0.000064,0.005141,0.000034,4,{'kneighborsclassifier__n_neighbors': 4},0.804348,0.673913,0.586957,...,0.695652,0.070107,5,0.755435,0.782609,0.788043,0.760870,0.782609,0.773913,0.013134


In [12]:
# plot the point with the best k-value
k_point = alt.Chart(cv_results.sort_values(by='rank_test_score').head(1).reset_index()).mark_point(filled = True).encode(
    x = alt.X('param_kneighborsclassifier__n_neighbors'),
    y = alt.Y('mean_test_score'),
    color = alt.Color('param_kneighborsclassifier__n_neighbors:N',title='Best K-Value').scale(scheme="set1")
)

# plot the cross-validation plot of all k values
cross_val_plot= alt.Chart(cv_results).mark_line(point=True).encode(
    x=alt.X("param_kneighborsclassifier__n_neighbors")
        .title('Number of Neighbors (k)'),
    y=alt.Y('mean_test_score')
        .title('Accuracy Score')
        .scale(zero=False)
).properties(title=['Fig 3. Cross Validation of K-Values from 1-30','Best K Marked in Red'])

cross_val_plot + k_point

alt.LayerChart(...)

In [19]:
# display the confusion matrix
best_k = cv_results.sort_values(by='rank_test_score').reset_index().loc[0,'param_kneighborsclassifier__n_neighbors']
players_spec = make_pipeline(players_preprocessor,KNeighborsClassifier(n_neighbors = best_k))

players_fit = players_spec.fit(players_train_upsample[['played_hours', 'age','experience_value']],players_train_upsample['subscribe'])
players_pred = players_fit.predict(players_test[['played_hours', 'age','experience_value']])
players_eval = players_test.assign(actual=players_test['subscribe'],predicted=players_pred)
players_conf_mat = pd.crosstab(players_eval['actual'], players_eval['predicted'])
players_conf_mat

predicted,False,True
actual,,
False,2,9
True,2,27


In [20]:
# report the measures of accuracy
print(classification_report(players_test['subscribe'], players_pred))

              precision    recall  f1-score   support

       False       0.50      0.18      0.27        11
        True       0.75      0.93      0.83        29

    accuracy                           0.72        40
   macro avg       0.62      0.56      0.55        40
weighted avg       0.68      0.72      0.68        40



### <u> **Discussion**

### <u> **References**

Pacific Laboratory for Artificial Intelligence. (n.d.). PLAI group website. Retrieved November 30, 2025, from https://plai.cs.ubc.ca/